In [ ]:
%cd ..
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import torch
import random
from utils import load_encoders
from models.sit import SiT_models
from diffusers import AutoencoderKL
import torch.nn.functional as F
import json
import numpy as np
import h5py
import os
import PIL
import json
import gc
import io
from torchvision.transforms import Normalize
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import matplotlib.pyplot as plt
from tqdm import tqdm
from metrics import AlignmentMetrics

pyspng = None

In [ ]:
### Prepare the data
with open("data/images_h5.json", "r") as f:
    images_h5_cfg = json.load(f)
with open("data/vae-sd_h5.json", "r") as f:
    vae_h5_cfg = json.load(f)

def load_h5_file(hf, path):
    # Helper function to load files from h5 file
    if path.endswith('.png'):
        if pyspng is not None:
            rtn = pyspng.load(io.BytesIO(np.array(hf[path])))
        else:
            rtn = np.array(PIL.Image.open(io.BytesIO(np.array(hf[path]))))
        rtn = rtn.reshape(*rtn.shape[:2], -1).transpose(2, 0, 1)
    elif path.endswith('.json'):
        rtn = json.loads(np.array(hf[path]).tobytes().decode('utf-8'))
    elif path.endswith('.npy'):
        rtn= np.array(hf[path])
    else:
        raise ValueError('Unknown file type: {}'.format(path))
    return rtn

def preprocess_raw_image(x, enc_type):
    x = x / 255.
    x = Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)(x)
    x = torch.nn.functional.interpolate(x, 224, mode='bicubic')
    x = torch.nn.functional.interpolate(x, 224, mode='bicubic')
    return x

# N = 256
N = 64
BS = 8
chosen_files = random.Random(42).sample(images_h5_cfg, N)
chosen_vaes = [elem.replace("img", "img-mean-std-").replace(".png", ".npy") for elem in chosen_files]

image_h5 = h5py.File("data/images.h5", "r")
vae_h5 = h5py.File("data/vae-sd.h5", "r")

### Labels...
fname = 'dataset.json'
labels = load_h5_file(vae_h5, fname)['labels']
labels = dict(labels)
labels = [labels[fname.replace('\\', '/')] for fname in chosen_vaes]
labels = np.array(labels)
labels = labels.astype({1: np.int64, 2: np.float32}[labels.ndim])

images = preprocess_raw_image(torch.stack([torch.from_numpy(load_h5_file(image_h5, elem)) for elem in chosen_files]), "dinov2-vit-b")
vaes = torch.stack([torch.from_numpy(load_h5_file(vae_h5, elem)) for elem in chosen_vaes])
labels = torch.from_numpy(labels)

In [ ]:
### Load external visual model, and extract features
encoders, encoder_types, architectures = load_encoders("dinov2-vit-b", "cuda:0")
# encoders, encoder_types, architectures = load_encoders("dinov2-vit-l", "cuda:0")
# encoders, encoder_types, architectures = load_encoders("dinov2-vit-g", "cuda:0")

encoder, encoder_type, architecture = encoders[0], encoder_types[0], architectures[0]
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema").to("cuda:0")

zs = []
for i in tqdm(range(0, N, BS)):
    images_batch = preprocess_raw_image(images[i:i+BS].to("cuda:0"), encoder_type)
    with torch.no_grad():
        z = encoder.forward_features(images_batch)['x_norm_patchtokens']
        zs.append(z)
zs = torch.cat(zs, dim=0).cpu()


# # NOTE: The model checkpoints we are interested in
# ckpt_paths = [
#     "exps/sit-b-base-400k/checkpoints", "exps/sit-b-linear-dinov2-b-enc8-400k/checkpoints",
#     # "exps/sit-b-linear-dinov2-b-enc8-patch-0.5-400k/checkpoints", "exps/sit-b-linear-dinov2-b-enc8-patch-0.75-400k/checkpoints",
# ]
# model_name = "SiT-B/2"

# ckpt_paths = ["exps/sit-xl-base-400k/checkpoints", "exps/sit-xl-linear-dinov2-b-enc8-400k/checkpoints"]
# model_name = "SiT-XL/2"

ckpt_paths = ["pretrained_models"]
model_name = "SiT-XL/2"


### Model HPs
resolution = 256
num_classes = 1000
assert resolution % 8 == 0
latent_size = resolution // 8
# z_dims = [encoder.embed_dim for encoder in encoders]
z_dims = [768]
encoder_depth = 8
block_kwargs = {"fused_attn": False, "qk_norm": False}


def get_model(ckpt_path):
    model = SiT_models[model_name](
        latent_size=latent_size,
        num_classes=num_classes,
        use_cfg=True,
        z_dims=z_dims,
        encoder_depth=encoder_depth,
        **block_kwargs,
    ).to("cuda:0")
    state_dict = torch.load(ckpt_path, map_location="cpu")
    if "model" in state_dict:
        state_dict = state_dict["model"]
    model.load_state_dict(state_dict)
    model.eval()
    return model

def interpolant(t):
    alpha_t = 1 - t
    sigma_t = t
    d_alpha_t = -1
    d_sigma_t = 1
    return alpha_t, sigma_t, d_alpha_t, d_sigma_t

def mean_flat(x):
    return torch.mean(x, dim=list(range(1, len(x.size()))))

@torch.no_grad()
def sample_posterior(moments, latents_scale=1., latents_bias=0.):
    mean, std = torch.chunk(moments, 2, dim=1)
    z = mean + std * torch.randn_like(mean)
    z = (z * latents_scale + latents_bias) 
    return z

latents_scale = torch.tensor(
    [0.18215, 0.18215, 0.18215, 0.18215]
).view(1, 4, 1, 1).to("cuda:0")
latents_bias = torch.tensor(
    [0., 0., 0., 0.]
).view(1, 4, 1, 1).to("cuda:0")

In [ ]:
ckpt_stats = {}

for ckpt_path in ckpt_paths:
    # stats_neg_cos_sim = {}
    # stats_frobenius_l2_norm = {}
    stats_cknna_align = {}
    stats_mutual_knn = {}
    stats_p2p_align = {}

    for elem in sorted(os.listdir(ckpt_path)):
        # Skip raw SiT checkpoints
        if elem == "SiT-XL-2-256.pt": continue
        model = get_model(os.path.join(ckpt_path, elem))

        # Call gc
        gc.collect()
        torch.cuda.empty_cache()

        zs_tilde = []
        for i in tqdm(range(0, N, BS)):
            vaes_batch = vaes[i:i+BS].to("cuda:0")
            labels_batch = labels[i:i+BS].to("cuda:0")
            x = sample_posterior(vaes_batch, latents_scale=latents_scale, latents_bias=latents_bias)
            model_kwargs = dict(y = labels_batch)
            # time_input = torch.rand((x.shape[0], 1, 1, 1), device="cuda:0", dtype=x.dtype)
            time_input = torch.rand((x.shape[0], 1, 1, 1), device="cuda:0", dtype=x.dtype)
            # time_input = torch.fill_(torch.zeros((x.shape[0], 1, 1, 1), device="cuda:0", dtype=x.dtype), 0.5)  # <--- control on time

            noises = torch.randn_like(x)
            alpha_t, sigma_t, d_alpha_t, d_sigma_t = interpolant(time_input)  # linear

            model_input = alpha_t * x + sigma_t * noises
            model_target = d_alpha_t * x + d_sigma_t * noises

            with torch.no_grad():
                model_output, z_tilde = model.forward_features(model_input, time_input.flatten(), **model_kwargs)
                # zs_tilde.append(z_tilde[0])
                zs_tilde.append(z_tilde)
        zs_tilde = torch.cat(zs_tilde, dim=0)

        cknna_align = []
        mutual_knn_align = []
        p2p_align = []
        for l in range(zs_tilde.shape[1]):
            # CKNNA method for alignment
            # print(zs_tilde[:, l].mean(dim=1).shape)
            # print(zs.mean(dim=1).shape)
            # break
            cknna = AlignmentMetrics.cknna(zs.mean(dim=1), zs_tilde[:, l].mean(dim=1), topk=10)
            mutual_knn = AlignmentMetrics.mutual_knn(zs.mean(dim=1), zs_tilde[:, l].mean(dim=1), topk=10)
            p2p = AlignmentMetrics.patch2patch_kernel_alignment_score(zs, zs_tilde[:, l])
            cknna_align.append(cknna)
            mutual_knn_align.append(mutual_knn)
            p2p_align.append(p2p)

        step = int(elem[:-3]) if elem != "last.pt" and elem != "SiT-XL-2-256_fixed.pt" else 7_000_000
        stats_cknna_align[step] = cknna_align
        stats_mutual_knn[step] =  mutual_knn_align
        stats_p2p_align[step] = p2p_align

    ckpt_stats[ckpt_path] = {"cknna_align": stats_cknna_align, "mutual_knn": stats_mutual_knn, "p2p_align": stats_p2p_align}
    # ckpt_stats[ckpt_path] = {"neg_cos_sim": stats_neg_cos_sim, "kernel_align_patch_l2": stats_frobenius_l2_norm, "cknna_align": stats_cknna_align, "mutual_knn": stats_mutual_knn}

In [ ]:
# ckpt_stats_path = "ckpt_stats_sit_b.json"
ckpt_stats_path = "ckpt_stats_sit_xl.json"

if os.path.exists(ckpt_stats_path):
    with open(ckpt_stats_path, "r") as f:
        ckpt_stats = json.load(f)

In [ ]:
print(ckpt_stats.keys())
print(ckpt_stats[ckpt_paths[0]].keys())
print(ckpt_stats[ckpt_paths[0]]["cknna_align"].keys())

In [ ]:
if ckpt_paths[0] == "pretrained_models":
    plt.figure(figsize=(10, 5))
    plt.plot(np.array(range(len(cknna_align))) + 1, cknna_align, label="CKNNA",marker='o')
    plt.plot(np.array(range(len(cknna_align))) + 1, mutual_knn_align, label="Mutual KNN", marker='x')
    plt.plot(np.array(range(len(cknna_align))) + 1, p2p_align, label="Patch2Patch", marker='s')
    plt.xlabel("Layer")
    plt.ylabel("Alignment score")
    plt.title("Alignment metrics for SiT-XL/2 model @ 7M steps")
    plt.xticks(np.array(range(len(cknna_align))) + 1)
    plt.ylim(0., 1.0)
    plt.legend()
    plt.show()

In [ ]:
### 3 plots for Figure 2 in the paper

# k = "exps/sit-b-base-400k/checkpoints"
# k = "exps/sit-b-linear-dinov2-b-enc8-400k/checkpoints"
k = 'exps/sit-xl-base-400k/checkpoints'
# k = 'exps/sit-xl-linear-dinov2-b-enc8-400k/checkpoints'

metrics = "cknna_align"
# metrics = "p2p_align"

if k in ckpt_paths:
    plt.figure(figsize=(10, 5))
    for step, align in ckpt_stats[k][metrics].items():
        plt.plot(np.array(range(len(align))) + 1, align, label=f"Step: {step}", marker='o')
    plt.xlabel("Layer")
    plt.ylabel(f"Alignment score ({metrics})")
    plt.title(k)
    plt.xticks(np.array(range(len(align))) + 1)
    plt.grid()
    plt.legend()
    plt.show()
    

# if "exps/sit-b-linear-dinov2-b-enc8-400k/checkpoints" in ckpt_paths:
#     plt.figure(figsize=(10, 5))
#     for step, align in ckpt_stats["exps/sit-b-linear-dinov2-b-enc8-400k/checkpoints"]["cknna_align"].items():
#         plt.plot(np.array(range(len(align))) + 1, align, label=f"Step: {step}", marker='o')
#     plt.xlabel("Layer")
#     plt.ylabel("Alignment score (CKNNA)")
#     plt.title("sit-b-linear-dinov2-b-enc8-400k")
#     plt.xticks(np.array(range(len(align))) + 1)
#     plt.grid()
#     plt.legend()
#     plt.show()

In [ ]:
# trend plot, x -> steps, y -> alignment score of a specific layer (e.g, 8th layer)

# k = "exps/sit-b-base-400k/checkpoints"
# k = "exps/sit-b-linear-dinov2-b-enc8-400k/checkpoints"
# k = 'exps/sit-xl-base-400k/checkpoints'
k = 'exps/sit-xl-linear-dinov2-b-enc8-400k/checkpoints'

enc_layer = 8
# enc_layer = 11
# enc_layer = -1

metrics = "cknna_align"
# metrics = "p2p_align"

if k in ckpt_paths:
    plt.figure(figsize=(10, 5))
    steps = []
    aligns = []
    for step, align in ckpt_stats[k][metrics].items():
        steps.append(step)
        if enc_layer == -1:
            aligns.append(np.max(align))
        else:
            aligns.append(align[enc_layer - 1])
    plt.plot(steps, aligns, label="Max" if enc_layer == -1 else f"Layer: {enc_layer}", marker='o')
    plt.xlabel("Step")
    plt.ylabel(f"Alignment score ({metrics})")
    plt.title(k)
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
if not os.path.exists(ckpt_stats_path):
    with open(ckpt_stats_path, "w") as f:
        json.dump(ckpt_stats, f)

---------------------------